In [219]:
import tweepy as tw

In [220]:
consumer_key = '3zw18AYLJBxmRCzC3gzN02GSP'
consumer_secret = 'ypRM5HD8vdW0WHq3j8gbYGnXw68GxCrED7QRqxYsHhvRRV54lZ'
access_token = "1133250449639915520-Uwmc9dRP9uSPooOetqdtCib7tlttWZ"
access_secret = 'CXiXO5KE89fypfJdSKpQXzS9OAXp6ijEkV50G2dwtOJcW'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [221]:
def determine_tweet_type(tweet):
    # Check for reply indicator first
    if tweet["in_reply_to_status_id"] is not None:
        tweet_type = "Reply Tweet"
    # Check boolean quote status field but make sure it's not a Retweet (of a Quote Tweet) 
    elif tweet["is_quote_status"] is True and not tweet["text"].startswith("RT"):
        tweet_type = "Quote Tweet"
    # Check both indicators of a Retweet
    elif tweet["text"].startswith("RT") and tweet.get("retweeted_status") is not None:
        tweet_type = "Retweet"
    else:
        tweet_type = "Original Tweet"
    return tweet_type 

In [222]:
def credible_source(user):
    
    if not user.description:
        return False
    
    if 'news' in user.description or 'coverage' in user.description:
        return True
    
    else:
        return False

In [223]:
def parse_tweets(status):
    
    parsedTweets = []
    
    for tweet in status:
        
        if determine_tweet_type(tweet._json) != 'Original Tweet':
            continue
            
        if not credible_source(tweet.user):
            continue
        
        else:
            if 'extended_tweet' in tweet._json:
                full_text = tweet._json['extended_tweet']['full_text']
            else:
                full_text = tweet._json['text']
        
        mydict = { "verified account": tweet.user.verified,
                   "account": tweet.user.screen_name,
                   "tweet_id": tweet._json["id_str"],
                   "date":tweet._json["created_at"],
                   "full_text": full_text,
                   "tweet_type": determine_tweet_type(tweet._json),
                   "hyperlink": "https://twitter.com/twitter/status/" + tweet._json["id_str"]
              }
        parsedTweets.append(mydict) # Add Tweet to parsedTweets list
    return parsedTweets

In [227]:
#edit keyword here
keyword = 'export ban'

In [225]:
retrieved = []
try:
    for page in tw.Cursor(api.search_30_day,label='GupiaoDev', 
                          query=keyword,
                          fromDate='202206200000').pages():
        
        parsed = parse_tweets(page)
        retrieved += parsed
        #print(parsed)
except:
    pass

In [226]:
retrieved

[{'verified account': True,
  'account': 'aajtak',
  'tweet_id': '1540341144189833216',
  'date': 'Fri Jun 24 14:28:41 +0000 2022',
  'full_text': 'भारत के गेहूं निर्यात प्रतिबंध से बांग्लादेश की बढ़ी मुश्किलें\n\n#India #Bangladesh #wheatexport \nhttps://t.co/NvCuWgl6Z2',
  'tweet_type': 'Original Tweet',
  'hyperlink': 'https://twitter.com/twitter/status/1540341144189833216'},
 {'verified account': True,
  'account': 'CNBCTV18Live',
  'tweet_id': '1540331350649376768',
  'date': 'Fri Jun 24 13:49:46 +0000 2022',
  'full_text': '#CNBCTV18Exclusive | Not considering a rice export ban, no more export curbs as things stand. A Trade promotion body will be set up by September, says B V R Subrahmanyam, Secretary, @DoC_GoI https://t.co/JlICylqy36',
  'tweet_type': 'Original Tweet',
  'hyperlink': 'https://twitter.com/twitter/status/1540331350649376768'},
 {'verified account': False,
  'account': 'rethink_link',
  'tweet_id': '1540201583849480193',
  'date': 'Fri Jun 24 05:14:07 +0000 2022',


In [214]:
with open('search result.txt','w') as writer:
    
    for tweet in retrieved:
        
        if not tweet['verified account']:
            continue
        
        writer.write('account:\n\t')
        writer.write(tweet['account'])
        writer.write('\n')
        writer.write('date:\n\t')
        writer.write(tweet['date'])
        writer.write('\n\n')
        
        writer.write('tweet text:\n\t')
        writer.write(tweet['full_text'])
        writer.write('\n\n')
        
        writer.write('link:\n\t')
        writer.write(tweet['hyperlink'])
        writer.write('\n\n')
        
        writer.write('\n\n\n\n\n')